# CAPSTONE PROJECT - STARBUCKS
# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

## Load Data

In [1]:
import pandas as pd
import numpy as np
import math
import json
import seaborn as sns
%matplotlib inline
sns.set(rc={'figure.figsize':(11.7,15)})

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [3]:
portfolio[['offer_type','id']]

,offer_type,id
0,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,informational,3f207df678b143eea3cee63160fa8bed
3,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,discount,fafdcd668e3743c1bb461111dcafc2a4
7,informational,5a8bc65990b245e5a138643cd4eb9837
8,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,discount,2906b810c7d4411798c6938adc9daaa5


In [4]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [5]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


# 1. Pre Processing
___
## 1.1 Data Cleaning

In [6]:
profile = profile.dropna(subset=['income'])

In [7]:
profile.head()

,gender,age,id,became_member_on,income
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
5,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0
8,M,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0


## 1.2 Data Preparation

In [8]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


### Transcript

In [9]:
transcript.event.unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

In [10]:
transcript[transcript.event == 'offer completed']

,person,event,value,time
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0
12672,fe97aa22dd3e48c8b143116a8403dd52,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,0
12679,629fc02d56414d91bca360decdfa9288,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0
12692,676506bad68e4161b9bbaffeb039626b,offer completed,{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...,0
12697,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0...,0
...,...,...,...,...
306475,0c027f5f34dd4b9eba0a25785c611273,offer completed,{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2...,714
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2...,714
306506,b895c57e8cd047a8872ce02aa54759d6,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,714
306509,8431c16f8e1d440880db371a68f82dd0,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,714


In [11]:
transactions = transcript[transcript.event == 'transaction']

In [12]:
transactions

,person,event,value,time
12654,02c083884c7d45b39cc68e1314fec56c,transaction,{'amount': 0.8300000000000001},0
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,{'amount': 34.56},0
12659,54890f68699049c2a04d415abc25e717,transaction,{'amount': 13.23},0
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,{'amount': 19.51},0
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,{'amount': 18.97},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


In [13]:
offer_events =  transcript[~(transcript.event == 'transaction')]

In [14]:
offer_events

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2...,714
306506,b895c57e8cd047a8872ce02aa54759d6,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,714
306507,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer viewed,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},714
306509,8431c16f8e1d440880db371a68f82dd0,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,714


In [15]:
offer_events['offer'] = [d.get('offer id') if d.get('offer id') is not None else (d.get('offer_id')) for d in offer_events.value]

/Users/luisacabs/Documents/capstone/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
offer_events = offer_events.drop(columns = ['value'])
offer_events

,person,event,time,offer
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0
...,...,...,...,...
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,714,2298d6c36e964ae4a3e7e9706d1fb8c2
306506,b895c57e8cd047a8872ce02aa54759d6,offer completed,714,fafdcd668e3743c1bb461111dcafc2a4
306507,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer viewed,714,0b1e1539f2cc45b7b9fa7c272da2e1d7
306509,8431c16f8e1d440880db371a68f82dd0,offer completed,714,fafdcd668e3743c1bb461111dcafc2a4


In [17]:
offer_events.offer.unique()

array(['9b98b8c7a33c4b65b9aebfe6a799e6d9',
       '0b1e1539f2cc45b7b9fa7c272da2e1d7',
       '2906b810c7d4411798c6938adc9daaa5',
       'fafdcd668e3743c1bb461111dcafc2a4',
       '4d5c57ea9a6940dd891ad53e9dbe8da0',
       'f19421c1d4aa40978ebb69ca19b0e20d',
       '2298d6c36e964ae4a3e7e9706d1fb8c2',
       '3f207df678b143eea3cee63160fa8bed',
       'ae264e3637204a6fb9bb56bc8210ddfd',
       '5a8bc65990b245e5a138643cd4eb9837'], dtype=object)

In [18]:
transactions['value'] = [d.get('amount') for d in transactions.value]

/Users/luisacabs/Documents/capstone/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
transactions

,person,event,value,time
12654,02c083884c7d45b39cc68e1314fec56c,transaction,0.83,0
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,34.56,0
12659,54890f68699049c2a04d415abc25e717,transaction,13.23,0
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,19.51,0
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,18.97,0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,1.59,714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,9.53,714
306531,a00058cf10334a308c68e7631c529907,transaction,3.61,714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,3.53,714


In [20]:
transactions.groupby('person').sum()#.sum('value')

,value,time
person,,
0009655768c64bdeb2e877511632db8f,127.60,4344
00116118485d4dfda04fdbaba9a87b5c,4.09,1224
0011e0d4e6b944f998e987f904e8c1e5,79.46,2256
0020c2b971eb4e9188eac86d93036a77,196.86,2790
0020ccbbb6d84e358d3414a3ff76cffd,154.05,4500
...,...,...
fff3ba4757bd42088c044ca26d73817a,580.98,2706
fff7576017104bcc8677a8d63322b5e1,29.94,2352
fff8957ea8b240a6b5e634b6ee8eafcf,12.15,1896


In [21]:
dir(transactions.groupby('person'))#.sum('value')

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accessors',
 '_agg_examples_doc',
 '_agg_general',
 '_aggregate_frame',
 '_aggregate_item_by_item',
 '_aggregate_with_numba',
 '_apply_allowlist',
 '_apply_filter',
 '_apply_to_column_groupbys',
 '_assure_grouper',
 '_bool_agg',
 '_builtin_table',
 '_choose_path',
 '_concat_objects',
 '_constructor',
 '_cumcount_array',
 '_cython_agg_blocks',
 '_cython_agg_general',
 '_cython_table',
 '_cython_transform',
 '_define_paths',
 '_dir_additions',
 '_dir_deletions',
 '_fill',
 

In [22]:
monetary_frequency = transactions.groupby('person').value.agg(['sum','count']).rename(columns = {'sum': 'monetary','count': 'frequency'})
monetary_frequency

,monetary,frequency
person,,
0009655768c64bdeb2e877511632db8f,127.60,8
00116118485d4dfda04fdbaba9a87b5c,4.09,3
0011e0d4e6b944f998e987f904e8c1e5,79.46,5
0020c2b971eb4e9188eac86d93036a77,196.86,8
0020ccbbb6d84e358d3414a3ff76cffd,154.05,12
...,...,...
fff3ba4757bd42088c044ca26d73817a,580.98,11
fff7576017104bcc8677a8d63322b5e1,29.94,6
fff8957ea8b240a6b5e634b6ee8eafcf,12.15,5


In [23]:
time_max = transactions.time.max()
recency = transactions.groupby('person').time.max().reset_index().rename(columns = {'time': 'recency'})
recency['recency'] = time_max - recency['recency']
recency

,person,recency
0,0009655768c64bdeb2e877511632db8f,18
1,00116118485d4dfda04fdbaba9a87b5c,240
2,0011e0d4e6b944f998e987f904e8c1e5,60
3,0020c2b971eb4e9188eac86d93036a77,6
4,0020ccbbb6d84e358d3414a3ff76cffd,42
...,...,...
16573,fff3ba4757bd42088c044ca26d73817a,162
16574,fff7576017104bcc8677a8d63322b5e1,18
16575,fff8957ea8b240a6b5e634b6ee8eafcf,138
16576,fffad4f4828548d1b5583907f2e9906b,36


In [24]:
rfm = transactions.merge(monetary_frequency, right_on='person',left_on = 'person')
rfm = rfm.merge(recency,right_on='person',left_on = 'person')
rfm = rfm[['person','recency','frequency','monetary']].drop_duplicates()
rfm = rfm.merge(profile, right_on='id',left_on='person')
rfm

,person,recency,frequency,monetary,gender,age,id,became_member_on,income
0,02c083884c7d45b39cc68e1314fec56c,294,10,29.89,F,20,02c083884c7d45b39cc68e1314fec56c,20160711,30000.0
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,174,12,320.48,M,42,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,20160117,96000.0
2,54890f68699049c2a04d415abc25e717,30,7,118.85,M,36,54890f68699049c2a04d415abc25e717,20171228,56000.0
3,b2f1cd155b864803ad8334cdf13c4bd2,144,8,195.35,F,55,b2f1cd155b864803ad8334cdf13c4bd2,20171016,94000.0
4,fe97aa22dd3e48c8b143116a8403dd52,6,11,562.77,F,39,fe97aa22dd3e48c8b143116a8403dd52,20171217,67000.0
...,...,...,...,...,...,...,...,...,...
14487,aabc3d56a2ea4446bef4c3f9eec8ab72,0,1,14.68,F,72,aabc3d56a2ea4446bef4c3f9eec8ab72,20180318,74000.0
14488,448dabde725040978b8a247a20bac126,0,1,3.19,M,50,448dabde725040978b8a247a20bac126,20180107,60000.0
14489,7718656997f3453db0f5aeca9cd35240,0,1,15.35,F,73,7718656997f3453db0f5aeca9cd35240,20180501,65000.0
14490,54463e5d95124b7fb3133fc1eae71952,0,1,0.76,M,42,54463e5d95124b7fb3133fc1eae71952,20171104,69000.0


In [25]:
people = rfm.person.unique()

In [26]:
transactions.groupby('person')

In [27]:
offer_events

,person,event,time,offer
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0
...,...,...,...,...
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,714,2298d6c36e964ae4a3e7e9706d1fb8c2
306506,b895c57e8cd047a8872ce02aa54759d6,offer completed,714,fafdcd668e3743c1bb461111dcafc2a4
306507,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer viewed,714,0b1e1539f2cc45b7b9fa7c272da2e1d7
306509,8431c16f8e1d440880db371a68f82dd0,offer completed,714,fafdcd668e3743c1bb461111dcafc2a4


In [28]:
len(offer_events[offer_events.event == 'offer received'])

76277

In [29]:
len(offer_events.person.unique())

16994

In [30]:
offer_events = offer_events[['person','event','offer']].drop_duplicates()
offer_events['val'] = int(1)

In [31]:
offer_acceptance = offer_events.pivot(index = ['person','offer'],columns = 'event',values = 'val').reset_index().fillna(0)

In [32]:
data = rfm.merge(offer_acceptance,left_on = 'person',right_on = 'person')

In [33]:
data_gender = pd.get_dummies(data.gender, prefix='gender')

In [34]:
data_gender

,gender_F,gender_M,gender_O
0,1,0,0
1,1,0,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
53982,0,1,0
53983,0,1,0
53984,0,1,0
53985,0,1,0


In [35]:
data = pd.concat([data, data_gender], axis=1).drop(columns = ['gender','person','id'])

In [36]:
data = data.merge(portfolio[['offer_type','id']], left_on = 'offer', right_on = 'id').drop(columns = ['offer','id'])

In [37]:
data

,recency,frequency,monetary,age,became_member_on,income,offer completed,offer received,offer viewed,gender_F,gender_M,gender_O,offer_type
0,294,10,29.89,20,20160711,30000.0,0.0,1.0,0.0,1,0,0,discount
1,174,12,320.48,42,20160117,96000.0,1.0,1.0,1.0,0,1,0,discount
2,30,7,118.85,36,20171228,56000.0,1.0,1.0,0.0,0,1,0,discount
3,48,3,66.05,52,20180605,72000.0,1.0,1.0,1.0,0,1,0,discount
4,108,8,121.86,75,20130806,69000.0,1.0,1.0,1.0,1,0,0,discount
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53982,6,1,0.69,37,20180308,40000.0,0.0,1.0,1.0,0,1,0,bogo
53983,6,1,16.85,69,20180704,85000.0,0.0,1.0,1.0,0,1,0,bogo
53984,0,1,25.37,51,20170827,78000.0,0.0,1.0,1.0,0,1,0,bogo
53985,0,1,19.02,62,20180302,75000.0,0.0,1.0,1.0,1,0,0,bogo


In [41]:
data.offer_type.unique()

array(['discount', 'bogo', 'informational'], dtype=object)

# 2. Model Training

In [50]:
import os

In [56]:
def make_csv(x, y, filename, data_dir):
    '''Merges features and labels and converts them into one csv file with labels in the first column.
       :param x: Data features
       :param y: Data labels
       :param file_name: Name of csv file, ex. 'train.csv'
       :param data_dir: The directory where files will be saved
       '''
    # make data dir, if it does not exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    data = np.insert(x,0,y,1)#np.hstack((x, np.transpose(y)))
    df = pd.DataFrame(data)
    df = df.dropna()
    print(df)
    df.to_csv(str(data_dir)+'/'+str(filename),index = False, header = False)
    #np.savetxt(str(data_dir)+'/'+str(filename), df, delimiter=",")
 
    
    # your code here
    
    
    # nothing is returned, but a print statement indicates that the function has run
    print('Path created: '+str(data_dir)+'/'+str(filename))

In [61]:
def make_csv(x,y,filename,data_dir):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    data = pd.concat([x, y], axis=1).dropna()
    data.to_csv(str(data_dir)+'/'+str(filename),index = False, header = False)

In [57]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
fake_x = [ [0.39814815, 0.0001, 0.19178082], 
           [0.86936937, 0.44954128, 0.84649123], 
           [0.44086022, 0., 0.22395833] ]

fake_y = [0, 1, 1]

make_csv(fake_x, fake_y, filename='to_delete.csv', data_dir='test_csv')

# read in and test dimensions
fake_df = pd.read_csv('test_csv/to_delete.csv', header=None)

# check shape
assert fake_df.shape==(3, 4), \
      'The file should have as many rows as data_points and as many columns as features+1 (for indices).'
# check that first column = labels
assert np.all(fake_df.iloc[:,0].values==fake_y), 'First column is not equal to the labels, fake_y.'
print('Tests passed!')

     0         1         2         3
0  0.0  0.398148  0.000100  0.191781
1  1.0  0.869369  0.449541  0.846491
2  1.0  0.440860  0.000000  0.223958
Path created: test_csv/to_delete.csv
Tests passed!


In [40]:
from sklearn.model_selection import train_test_split

In [42]:
bogo = data[data.offer_type == 'bogo'].drop(columns = ['offer_type'])
info = data[data.offer_type == 'informational'].drop(columns = ['offer_type'])
disc = data[data.offer_type == 'discount'].drop(columns = ['offer_type'])

In [45]:
bogo_y = bogo['offer completed']
bogo_X = bogo.drop(columns = ['offer completed'])
bogo_X_train, bogo_X_test, bogo_y_train, bogo_y_test = train_test_split(bogo_X, bogo_y, test_size=0.33, random_state=42)

In [46]:
info_y = info['offer completed']
info_X = info.drop(columns = ['offer completed'])
info_X_train, info_X_test, info_y_train, info_y_test = train_test_split(info_X, info_y, test_size=0.33, random_state=42)

In [47]:
disc_y = disc['offer completed']
disc_X = disc.drop(columns = ['offer completed'])
disc_X_train, disc_X_test, disc_y_train, disc_y_test = train_test_split(disc_X, disc_y, test_size=0.33, random_state=42)

In [53]:
bogo_X_train

,recency,frequency,monetary,age,became_member_on,income,offer received,offer viewed,gender_F,gender_M,gender_O
51503,78,4,10.46,24,20180407,39000.0,1.0,1.0,0,1,0
9463,30,18,66.34,40,20131008,56000.0,1.0,1.0,1,0,0
52258,54,7,129.48,40,20150816,74000.0,1.0,1.0,1,0,0
46060,0,11,230.57,73,20151217,91000.0,1.0,1.0,1,0,0
9395,366,2,60.48,45,20180422,96000.0,1.0,0.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
44369,408,3,85.32,59,20171230,102000.0,1.0,0.0,0,1,0
53980,12,1,2.53,65,20180521,63000.0,1.0,1.0,0,1,0
10841,126,1,20.16,51,20180611,77000.0,1.0,1.0,1,0,0
6311,84,13,189.00,84,20170425,61000.0,1.0,1.0,1,0,0


In [54]:
bogo_y_train

51503    0.0
9463     0.0
52258    1.0
46060    1.0
9395     0.0
        ... 
44369    0.0
53980    0.0
10841    0.0
6311     1.0
48200    0.0
Name: offer completed, Length: 14459, dtype: float64

In [64]:
data_dir = 'model_data/bogo'
make_csv(bogo_X_train, bogo_y_train, filename='train.csv', data_dir=data_dir)
make_csv(bogo_X_test, bogo_y_test, filename='test.csv', data_dir=data_dir)

In [65]:
data_dir = 'model_data/info'
make_csv(info_X_train, info_y_train, filename='train.csv', data_dir=data_dir)
make_csv(info_X_test, info_y_test, filename='test.csv', data_dir=data_dir)

In [66]:
data_dir = 'model_data/disc'
make_csv(disc_X_train, disc_y_train, filename='train.csv', data_dir=data_dir)
make_csv(disc_X_test, disc_y_test, filename='test.csv', data_dir=data_dir)